In [1]:
import os
import pandas as pd
os.chdir('/home/gaojie/workspace/Mida_collab/')

In [2]:
links = pd.read_csv('CellOracle/output/links_filtered.csv',index_col=0,header=0)
base_GRN = pd.read_parquet('CellOracle/input/base_GRN_dataframe.parquet')
gene2peak = pd.read_csv('Multi-omics_intergration/output/gene2peak_df_all.csv',index_col=0)
gradient_gene = pd.read_csv('CellOracle/gradient_table2.vRG.tsv',sep='\t')
gradient_gene = gradient_gene[gradient_gene['flag']=='Pass'] ### check pass

In [3]:
base_GRN.head()

,peak_id,gene_short_name,qval,score,pval,9430076C15RIK,AC002126.6,AC012531.1,AC226150.2,AFP,...,ZNF784,ZNF8,ZNF816,ZNF85,ZSCAN10,ZSCAN16,ZSCAN22,ZSCAN26,ZSCAN31,ZSCAN4
0,chr10_100009027_100009527,DNMBP,0.006143,0.806778,0.002391,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,chr10_100009744_100010244,DNMBP,0.011732,0.681134,0.009856,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,chr10_100910568_100911068,SLF2,0.006082,0.931589,0.000568,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,chr10_100912022_100912522,SLF2,0.006082,0.925895,0.000636,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,chr10_100912609_100913109,SLF2,0.014486,0.655965,0.012746,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### calculate gradient gene score

In [4]:
print(f'max value in gene2peak score is {gene2peak["score"].max()}')
print(f'min value in gene2peak score is {gene2peak["score"].min()}')
print(f'make sure all gene-peak association is binarized {base_GRN.iloc[:,6:].isin([0,1]).all().all()}')
gene2peak = gene2peak[gene2peak['qval']<=0.05]
gene2peak = gene2peak[gene2peak['pval']<=0.05]

max value in gene2peak score is 1.000000238418579
min value in gene2peak score is 0.5107951164245605
make sure all gene-peak association is binarized True


In [5]:
###match format
base_GRN = base_GRN[base_GRN['qval']<=0.05]
gene2peak['peak_id'] = [i.replace(':','_') for i in gene2peak['peak_id']] 
gene2peak['peak_id'] = [i.replace('-','_') for i in gene2peak['peak_id']]
gene2peak['peak_gene'] = gene2peak['peak_id']+'_'+gene2peak['gene_short_name']
base_GRN['peak_gene'] = base_GRN['peak_id']+'_'+base_GRN['gene_short_name']
regulon = base_GRN.merge(gene2peak,left_on='peak_gene',right_on='peak_gene',how='inner')
print(regulon.head(5))

                   peak_id_x gene_short_name_x    qval_x   score_x    pval_x  \
0  chr10_100009027_100009527             DNMBP  0.006143  0.806778  0.002391   
1  chr10_100009744_100010244             DNMBP  0.011732  0.681134  0.009856   
2  chr10_100910568_100911068              SLF2  0.006082  0.931589  0.000568   
3  chr10_100912022_100912522              SLF2  0.006082  0.925895  0.000636   
4  chr10_100912609_100913109              SLF2  0.014486  0.655965  0.012746   

   9430076C15RIK  AC002126.6  AC012531.1  AC226150.2  AFP  ...  ZSCAN22  \
0              0           0           0           0    0  ...        0   
1              0           0           0           0    0  ...        0   
2              0           0           0           0    0  ...        0   
3              0           0           0           0    0  ...        0   
4              0           0           0           0    0  ...        0   

   ZSCAN26  ZSCAN31  ZSCAN4                        peak_gene  \
0   

In [6]:
###convert matrix to a long format df
long_data = []
for tf in regulon.columns[6:-6]:  
    tf_data = regulon[regulon[tf] == 1][['peak_id_x','peak_id_y', 'gene_short_name_x','gene_short_name_y','qval_x','qval_y','score_y','pval_y']]
    tf_data['TF'] = tf
    long_data.append(tf_data)
long_df = pd.concat(long_data, ignore_index=True)
long_df = long_df[['TF', 'peak_id_x','peak_id_y', 'gene_short_name_x','gene_short_name_y','qval_x','qval_y','score_y','pval_y']]
print(long_df.head(5))

           TF                  peak_id_x                  peak_id_y  \
0  AC002126.6  chr10_103919201_103919701  chr10_103919201_103919701   
1  AC002126.6  chr10_104587708_104588208  chr10_104587708_104588208   
2  AC002126.6  chr10_104599873_104600373  chr10_104599873_104600373   
3  AC002126.6  chr10_104707958_104708458  chr10_104707958_104708458   
4  AC002126.6  chr10_104809698_104810198  chr10_104809698_104810198   

  gene_short_name_x gene_short_name_y    qval_x    qval_y   score_y    pval_y  
0              STN1              STN1  0.006082  0.006082  0.893308  0.000894  
1        AL161646.1        AL161646.1  0.008425  0.008425  0.725277  0.006210  
2        AL161646.1        AL161646.1  0.006082  0.006082  0.835030  0.001720  
3            SORCS3            SORCS3  0.006082  0.006082  0.885200  0.001015  
4            SORCS3            SORCS3  0.006252  0.006252  0.801448  0.002597  


In [7]:
long_df['TF_gene'] = long_df['TF']+'_'+long_df['gene_short_name_x']
links['source_target'] = links['source']+'_'+links['target']
network_with_peaks = links.merge(long_df,how='left',left_on='source_target',right_on='TF_gene')
network_with_peaks = network_with_peaks[network_with_peaks['coef_mean']>0]

In [8]:
network_with_peaks['TF_gene_score'] = network_with_peaks['coef_mean']*network_with_peaks['score_y'] ###we use coef_mean multiple peak score to represent regulation intensity
TF_gene_score = network_with_peaks.groupby(by='TF_gene')['TF_gene_score'].mean().reset_index().reset_index(drop=True) ###mean score stand for the whole TF-peak-gene regulation
TF_gene_score.index = TF_gene_score['TF_gene']
TF_gene_score.index.name = None
TF_gene_score['TF'] = TF_gene_score['TF_gene'].apply(lambda x: x.split('_')[0])
TF_gene_score['gene'] = TF_gene_score['TF_gene'].apply(lambda x: x.split('_')[1])
TF_gene_score = TF_gene_score[['TF','gene','TF_gene_score']]

In [9]:
TF_gene_score.head()

,TF,gene,TF_gene_score
AHR_ABHD15-AS1,AHR,ABHD15-AS1,0.001484
AHR_AC002463.1,AHR,AC002463.1,0.002960
AHR_AC006387.1,AHR,AC006387.1,0.001993
AHR_AC007344.1,AHR,AC007344.1,0.000485
AHR_AC007389.1,AHR,AC007389.1,0.000994


In [10]:
TF_gene_score.to_csv('CellOracle/TF_gene_score.csv')